In [1]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#ALL USED FUNCTIONS


#VISUALIZATION
# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray,name):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img,name=name)

# visualize different images in the same moment
def visualize_all_list_napari(numpy_img_list: np.ndarray,names):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for i, img in enumerate(numpy_img_list):
            viewer.add_image(img, name=names[i] )



#PREPROCESSING
def sharpen(image: np.ndarray):
    """
    Sharpen the image
    :param image: image to be sharpened
    :return: sharp image
    """
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    return cv.filter2D(image, -1, kernel)




#THRESHOLDING

# compute Otsu's thresholding
def otsu_thresholding(image : np.ndarray):
    """
    Threshold and binarize an image using Otsu's method

    :param image: image you want to threshold
    :return: ret: threshold value
              th: binary image
    """
    ret,th = cv.threshold(image,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    #ret is the computed value of the threshold AND th is the image with the threshold applied
    return ret, th


# split the whole images into tiles
def split_into_tiles(image : np.ndarray, tile_size: int):
    """
    split image into tiles of shape tile_size*tile_size

    :param image: image to be split
    :param tile_size: dimensions of single tiles
    :return: tiles: list with the different tiles
    """
    tiles = []
    for i in range(0, image.shape[0], tile_size):
        for j in range(0, image.shape[1], tile_size):
            tile = image[i:i+tile_size, j:j+tile_size]
            tiles.append(tile)
    return tiles

#reconstruct image from different tiles given the number of tiles in x and y direction and a list of tiles
def reconstruct_image(tiles: list, x_tiles: int, y_tiles: int):
    """
    :param tiles:    list with the different single tiles
    :param x_tiles:  how many tiles fit in the x axis
    :param y_tiles:  how many tiles fit in the y axis
    :return:         numpy array, reconstructed image
    """
    big_image = np.zeros((x_tiles*tiles[0].shape[0], y_tiles*tiles[0].shape[1]))
    for i in range(x_tiles):
        for j in range(y_tiles):
            big_image[i*tiles[0].shape[0]:(i+1)*tiles[0].shape[0], j*tiles[0].shape[1]:(j+1)*tiles[0].shape[1]] = tiles[i*y_tiles+j]
    return big_image

def otsu_split_thresholding(img: np.ndarray, tile_size = 16):
    """
    Perform Otsu tresholding on sub images of 16 x 16,
    if a tile is all white do not apply otsu

    :param img:  image to be
    :return thresholded_tiles_sharp: list with thresholded tiles, to be recomposed
    """
    #sharpen image
    sharpened_img = sharpen(img)
    #get the maximum of the sharpened img, needed to check ig image is all white
    max_value=sharpened_img.max()
    #split
    tiles_sharpened=split_into_tiles(sharpened_img, tile_size)
    #do thresholding
    thresholded_tiles_sharp=[]
    for t in tiles_sharpened:
        #check if mostly white #ATTENTION, im touching the og in memory. If yes set direclty to black
        if check_all_white_tile(t,max_value):
            th=set_zero(t)
            thresholded_tiles_sharp.append(th)
        #else do thresholding
        else:
            r,th=otsu_thresholding(t)
            thresholded_tiles_sharp.append(th)

    return thresholded_tiles_sharp



def otsu_cleaned_split_thresholding(img):
    """
    Performe otsu thresholding on 16 x 16 images, then clean the image,
     delete the noise

    :param img: image to be tresholded
    :return:    tresholded clean image
    """

    #list with the thresholded tiles size 16x16
    thresholded_tiles = otsu_split_thresholding(img,16)




    #clean
    cleaned_tiles=[]
    for tl in thresholded_tiles:
        #check if image is not a bacilli
        if check_image(tl):
            #im not a bacilli
            m=set_zero(tl)
            cleaned_tiles.append(m)
        else:
            #i am a bacilli
            cleaned_tiles.append(tl)

    #reconstruct
    reconstructed_clean_image=reconstruct_image(cleaned_tiles,128,94)

    #final cleaning
    #final_cleaned_image = (reconstructed_clean_image)

    #visualize
    #visualize_all_list_napari([reconstructed_image, reconstructed_clean_image,sharpened_img,img], ["reconstructed_image","reconstructed_clean_image","sharpened_img","img"])

    return reconstructed_clean_image


#POSTPROCESSING

def check_image(img: np.ndarray):
    """
    For every sub-image we check if its worth keeping or not
    215 pretty hard-coded---> maybe rely on scientific paper to find the optimal number

    :param img: image to be checked
    :return: bool
    """

    if np.sum(img == 0) > 215: #we have a bacilli
        return False
    else:
        return True

#set pixels that are 255 to zero (black)
def set_zero(img):
    h=img
    h[h>0]=0
    return h


def check_all_white_tile(img,max_value_global):
    """
    Check if we have a huge bright tile. if a 16 x 16 tile is all white--->
    we want it black. Check based on global max pixel value

    :param img: tile to be checked if white
    :param max_value: max value pixel of whole image
    :return:
    """
    if np.sum(img > 0.2* max_value_global) > 0.9*img.shape[0]*img.shape[1]:
        return True
    else:
        return False

def clean_connected_components(img: np.ndarray):
    """
    Clean image with 2 approaches: delete connected components that have are up to 2 pixels
                                   connect bacilli that are separated by just one black pixel

    :param img: image to be cleaned
    :return:    cleaned image
    """

    #find connected components
    num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(np.uint8(img), connectivity=8)
    #stats = x,y,w,h,area

    #put to black connected components which area is equal to 1 or 2
    for i in range(1, num_labels):
        if stats[i][4] < 3:
            img[labels_im == i] = 0
    #do not want to connect bacilli in original, want to connect after little components are gone
    img2=img.copy()
    #connect the bacilli, by putting a white tile
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            if img[i,j] == 0:
                if (img[i-1,j] == 255 and img[i+1,j] == 255) or (img[i,j-1] == 255 and img[i,j+1] == 255) \
                        or (img[i-1,j-1] ==255 and img[i+1,j+1]) or (img[i-1,j+1] == 255 and img[i+1,j-1] == 255) \
                        or (img[i-1,j]== 255 and img[i+1,j+1]==255) or (img[i-1,j+1]==255 and img[i+1,j]==255)\
                        or (img[i-1,j]==255 and img[i+1,j-1]==255) or (img[i-1,j-1]==255 and img[i+1,j]==255)\
                        or (img[i,j-1]==255 and img[i+1,j+1]==255) or (img[i,j-1]==255 and img[i-1,j+1]==255)\
                        or (img[i,j+1]==255 and img[i+1,j-1]==255) or (img[i,j+1]==255 and img[i-1,j-1]==255):
                    img2[i,j] = 255

    return img2

# add the 2d bounding boxes to the image
def add_bounding_boxes(image, stats):
    """
    Add white rectangles around bacilli, based on conected components

    :param image: image with bacilli to be boxed
    :param coordinates:  coordinates of the center of the bacillus
    """
    for i in range(1,len(stats)):
            x = stats[i][0] - 5
            #x_max = coordinates[i][0]
            y = stats[i][1] - 5
            #y_max = coordinates[i][1]
            h=stats[i][3]
            w=stats[i][2]
            cv.rectangle(image, (x, y), (x+w+10, y+h+10), (5000, 255, 255), 1)
    return image

In [32]:
def sharpen2(img):
    return cv.addWeighted(img, 4, cv.blur(img, (30, 30)), -4, 128)

In [3]:
#PIPELINE (until the end)

# Load the image
reader = CziReader("TB_sample\extern_Synlab_2156_17_3_MTB.czi")
# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [4]:
def is_blurry(image):
    #compute laplacian of image
    laplacian = cv.Laplacian(image, cv.CV_64F)
    #compute variance of laplacian
    measure=laplacian.var()/(np.mean(image))
    #print(measure)
    return measure

In [18]:
print(type(smear))
mes=[]
for tile in smear:
    re=is_blurry(tile)
    mes.append(re)

#get mean of list
g=np.array(mes)
s=np.sort(g)
f=g.argsort()
a=smear[f[0]]
b=smear[f[1]]
new_smear =  np.stack((a,b), axis=0)




print(smear.shape)
print(new_smear.shape)

for i in range(2,g.shape[0]):
    print(i)
    if i % 11 ==0:
       new_smear = np.concatenate((new_smear, [smear[f[i]]]), axis= 0)



<class 'numpy.ndarray'>
(1345, 2048, 1504)
(2, 2048, 1504)
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263

In [19]:
 print(new_smear.shape)
for i in range(0,len(s),10):
  print(s[i]  )
visualize_napari(new_smear,'ff')



(124, 2048, 1504)
66.25414263681792
89.71273171783196
97.91733328503642
105.22798458705155
107.92495031303163
109.27225900983272
109.5807952270254
109.64793838096985
109.6691118153768
109.70391125104352
109.72216833775985
109.73492449349563
109.75660571650107
109.76467811104939
109.77382080004271
109.78299785791202
109.79151946210142
109.80326938845403
109.80946654719703
109.82128842679609
109.8286437887224
109.83639179179357
109.84835374442028
109.85610011801298
109.8672908543245
109.87912254422388
109.88415379820643
109.89174727543394
109.8997893737215
109.90499278177155
109.90989604867747
109.91992800272583
109.92629662734046
109.93205605692175
109.93962955662754
109.94712290636197
109.95420525296603
109.96127403335792
109.96652476474047
109.97467070123155
109.979711033972
109.98556873106494
109.99867867298111
110.00251117766763
110.01170020984014
110.02021223199273
110.0251488317866
110.03494551631296
110.03830388503306
110.04795694597989
110.05339662344251
110.06431383254882
110.0

C:\Users\matteo\anaconda3\envs\tub\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(


In [17]:
print(s[880])

110.63716823745895


In [20]:
# save in a new variable the information regarding the 673th tile out of 1345
img = smear[673]
print(img.shape)
real_image=img.copy()

(2048, 1504)


In [18]:
#for comparison
_, otsu_thresholded_entire_og_img=otsu_thresholding(img)
sharpened_img=sharpen(img)
_, otsu_thresholded_entire_sharp_img = otsu_thresholding(sharpened_img)


In [5]:
#do split thresholding
otsu_st_16=reconstruct_image(otsu_split_thresholding(img,16),128,94)

In [6]:
#do split thresholding and clean noise from thresholding
otsu_st_16_cleaned_from_noise = otsu_cleaned_split_thresholding(img)

In [7]:
#save new copy for comparison, not needed if we dont want comparison
a_bit_of_shit_in_our_bacilli_tiles = otsu_st_16_cleaned_from_noise .copy()

In [8]:
#cut small connected components and connect bacilli
cleaning_the_shit_on_the_bacilli_tiles = clean_connected_components(otsu_st_16_cleaned_from_noise)

In [9]:
#get stats for drawing boxes
num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(np.uint8(cleaning_the_shit_on_the_bacilli_tiles), connectivity=8)

In [10]:
#another copy----ask marina
bacilli =otsu_st_16_cleaned_from_noise.copy()

In [11]:
#add the boxes
box_bacilli = add_bounding_boxes(bacilli, stats)

In [12]:
img_copy=img.copy()
box_img=add_bounding_boxes(img_copy,stats)

In [13]:
#visualize
visualize_all_list_napari([box_img,box_bacilli,cleaning_the_shit_on_the_bacilli_tiles,a_bit_of_shit_in_our_bacilli_tiles,otsu_st_16_cleaned_from_noise,otsu_st_16,img],["og boxes","boxes","no 1s and unite bacilli", "a bit of artifacts","cleaned split otsu","split otsu","og"])

C:\Users\matteo\anaconda3\envs\tub\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
21-Dec-22 17:02:57 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 2172x1352+0+20 (frame: 2194x1408-11-25) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1924x1335+3+34 (frame: 1946x1391-8-11) margins: 11, 45, 11, 11 minimum size: 374x552 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=770,1160 maxtrack=0,0)


In [16]:
#fucntion for adding paadding to the image
def add_padding(image):

    image = np.pad(image, 8, 'edge')
    return image